In [15]:
import os
from langchain.chat_models import ChatGooglePalm
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import (
    ConversationBufferMemory, ConversationSummaryMemory, ConversationBufferWindowMemory
)

In [3]:
llm = ChatGooglePalm(
    google_api_key=os.getenv("PALM_API_KEY")
)

f:\Jupyter\Upwork\DocumentQA\DocumentQA\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### ConversationChain
The simplest form of memory, passing entire previous conversation as history.

In [4]:
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [5]:
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': 'Good morning to you too! How can I help you today?'}

In [8]:
conversation_buf.memory.buffer

'Human: Good morning AI!\nAI: Good morning to you too! How can I help you today?'

### ConversationSummaryMemory
Useful as it summarizes the history before passing it as a history variable in the prompt.

#### Benefits
1. This reduces information passed to the llm to prevent exceeding the context window.
2. It also reduces token usage on each api call which reduces costs.

In [12]:
conversation_sum = ConversationChain(
	llm=llm,
	memory=ConversationSummaryMemory(llm=llm)
)

In [13]:
conversation_sum("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': 'Good morning to you too! How can I help you today?'}

In [14]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


### ConversationBufferWindowMemory
In this memory we only keep a given number of past interactions before “forgetting” them, as defined by the `k` parameter.

In [17]:
conversation_bufw = ConversationChain(
	llm=llm,
	memory=ConversationBufferWindowMemory(k=5)
)

In [18]:
conversation_bufw("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': 'Good morning to you too! How can I help you today?'}

Buffer window exhibits `effective memory`, ie, what is actually available to the model.

In [19]:
conversation_bufw.memory.load_memory_variables(
    inputs=[]
)['history']

'Human: Good morning AI!\nAI: Good morning to you too! How can I help you today?'

### Summary
From empirical results, at `k=6` we can achieve lower token usage (`~ 1.5 tokens `) per call than any of the previous memory types.

The choice is then to use:

- ConversationBufferMemory:     for lower values of k, under 10
- ConversationSummaryMemory:    where longer recall is required; even the entire previous conversation.